In [1]:
# import libraries
from vpython import *
import numpy as np
import pandas as pd
import robotics_library as rbl

In [2]:
link_lengthes = [1, 3, 1, 1, 1, 1, 1]
rm = rbl.Robotic_Manipulator_Naive(link_lengthes)

In [3]:
qs = [0] * 6
# qs[0] = np.pi / 6
joint_locs = rm.loc_joints(qs)
# print(joint_locs[:, :3])

In [23]:
def draw_robotics(joint_locs):
    scene = canvas()
    # plot the axises
    axis_length = 3
    x = arrow(pos=vector(0,0,0), axis=vector(axis_length,0,0), shaftwidth=0.1, color=color.red)
    text(text='x', pos=vector(axis_length,0,0), axis=vector(axis_length,0,0))
    y = arrow(pos=vector(0,0,0), axis=vector(0,axis_length,0), shaftwidth=0.1, color=color.green)
    text(text='y', pos=vector(0,axis_length,0), axis=vector(0,axis_length,0))
    z = arrow(pos=vector(0,0,0), axis=vector(0,0,-axis_length), shaftwidth=0.1, color=color.blue)
    text(text='-z', pos=vector(0,0,-axis_length), axis=vector(0,0,-axis_length))
    
    # color list
    color_list = [color.cyan, color.yellow, color.orange, color.magenta, vector(0.7, 0.2, 0.1),
                  vector(0.2, 0.1, 0.7), vector(0.7, 0.3, 0.8)]
    
    # draw the links
    c = curve(pos=[vector(0, 0, 0), vector(*joint_locs[0, :3])], radius=0.5)
    pts = []
    for idx in xrange(1, len(joint_locs), 1):
        p0 = vector(*joint_locs[idx-1, :3])
        p1 = vector(*joint_locs[idx, :3])
        c = curve(pos=[p0, p1], color=color_list[idx])
        
    box(pos=vector(*joint_locs[-1, :3]), length=0.7, height=0.2, width=0.3)
    display()

In [28]:
qs = [np.pi/6, np.pi/4, 0, 0, 0, 0]
rm.configure_robots(qs)
draw_robotics(rm.joint_abs_locations)

<IPython.core.display.Javascript object>